In [1]:
import pandas as pd
from decimal import Decimal
import json
import math
import random
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-5

In this demo we investigate the **Fast Router performance** against test orders

### Current Issues
- tradeBySource orderIndex=1 example 53

Initialize a fast simulator

In [2]:
filename = '20221219_tradeBySourceAmount.json'
if 'Target' in filename:
    is_by_target = True
else:
    is_by_target = False
f = open(filename , "r")
bm = f.read()
bm_json = json.loads(bm)
print(len(bm_json))

120


In [3]:
def decimalize_dict(dicta):
    return({k:Decimal(v) for k,v in dicta.items()})

def calc_y_int(liquidity, highestRate, lowestRate, marginalRate):
    return(liquidity * (highestRate.sqrt() - lowestRate.sqrt()) / (marginalRate.sqrt() - lowestRate.sqrt()))

def parse_order(order0, order_count):
    order0 = decimalize_dict(order0)
    order0['delta_liquidity'] = order0['newLiquidity'] - order0['liquidity']
    # order0['y_int'] = calc_y_int(order0['liquidity'], order0['highestRate'], order0['lowestRate'], order0['marginalRate'])
    # order0['fix_trade'] = order0['y_int'] - order0['liquidity']
    # order0['y_int_minus_fixtrade'] = order0['y_int'] - order0['fix_trade']
    order0['order'] = order_count
    order0_df = pd.DataFrame.from_dict(order0, 'index', columns=[f'{order_count}'])
    return(order0_df)

def parse_strategy(strategy, strat_count, order_count):
    orders = pd.DataFrame()
    for order in strategy:
        order_df = parse_order(order, order_count)
        orders = pd.concat([orders,order_df.T])
        order_count += 1
    # orders['pair'] = [Sim.carbon_pair.pair_slash, Sim.carbon_pair.reverse.pair_slash]
    orders['strategy'] = strat_count
    strat_count += 1
    return(orders, strat_count, order_count)


In [4]:
len(bm_json)

120

In [5]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

setup_index = 0 #random.randint(0, len(bm_json))
print(setup_index)
strats = pd.DataFrame()
strat_count = 0
order_count = 0
for strategy in bm_json[setup_index]['strategies']:
    orders, strat_count, order_count = parse_strategy(strategy, strat_count, order_count)
    strats = pd.concat([strats, orders])
strats

0


,liquidity,lowestRate,highestRate,marginalRate,newLiquidity,newMarginalRate,delta_liquidity,order,strategy
0,864192,8.000000000000,9.000000000000,9.000000000000,865192,9.000000000000,1000,0,0
1,2962962,1.000000000000,2.000000000000,2.000000000000,2960963,1.999209661553,-1999,1,0
2,987648,9.000000000000,10.000000000000,10.000000000000,987648,10.000000000000,0,2,1
3,3950616,2.000000000000,3.000000000000,3.000000000000,3950616,3.000000000000,0,3,1
4,1111104,10.000000000000,11.000000000000,11.000000000000,1111104,11.000000000000,0,4,2
5,4938270,3.000000000000,4.000000000000,4.000000000000,4938270,4.000000000000,0,5,2
6,1234560,11.000000000000,12.000000000000,12.000000000000,1234560,12.000000000000,0,6,3
7,5925924,4.000000000000,5.000000000000,5.000000000000,5925924,5.000000000000,0,7,3
8,1358016,12.000000000000,13.000000000000,13.000000000000,1358016,13.000000000000,0,8,4
9,6913578,5.000000000000,6.000000000000,6.000000000000,6913578,6.000000000000,0,9,4


In [6]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

for i in strats.order[::2]:
    FastSim.add_strategy(
        tkn='USDC',
        amt_sell=strats.liquidity[i],
        psell_start=strats.highestRate[i],
        psell_end=strats.lowestRate[i],
        amt_buy=strats.liquidity[i+1],
        pbuy_start=1/strats.highestRate[i+1],
        pbuy_end=1/strats.lowestRate[i+1],
        psell_marginal=strats.marginalRate[i],
        pbuy_marginal=1/strats.marginalRate[i+1],
    )
initial_orders = FastSim.state()['orders']
initial_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,864192.00000,864192.00000,USDC,False,9.00000,8.00000,9.00000,USDC per ETH,1
1,1,ETHUSDC,ETH,2962962.00000,2962962.00000,ETH,False,0.50000,1.00000,0.50000,USDC per ETH,0
2,2,ETHUSDC,USDC,987648.00000,987648.00000,USDC,False,10.00000,9.00000,10.00000,USDC per ETH,3
3,3,ETHUSDC,ETH,3950616.00000,3950616.00000,ETH,False,0.33333,0.50000,0.33333,USDC per ETH,2
4,4,ETHUSDC,USDC,1111104.00000,1111104.00000,USDC,False,11.00000,10.00000,11.00000,USDC per ETH,5
5,5,ETHUSDC,ETH,4938270.00000,4938270.00000,ETH,False,0.25000,0.33333,0.25000,USDC per ETH,4
6,6,ETHUSDC,USDC,1234560.00000,1234560.00000,USDC,False,12.00000,11.00000,12.00000,USDC per ETH,7
7,7,ETHUSDC,ETH,5925924.00000,5925924.00000,ETH,False,0.20000,0.25000,0.20000,USDC per ETH,6
8,8,ETHUSDC,USDC,1358016.00000,1358016.00000,USDC,False,13.00000,12.00000,13.00000,USDC per ETH,9
9,9,ETHUSDC,ETH,6913578.00000,6913578.00000,ETH,False,0.16667,0.20000,0.16667,USDC per ETH,8


In [7]:
FastSim.state()['trades']

""


In [8]:
bm_json[setup_index]['actions']

[{'strategyId': 0, 'orderIndex': 0, 'tokenAmount': '1000'}]

In [9]:
actionList = bm_json[setup_index]['actions']
    
print(f'is_by_target {is_by_target}')
for action in actionList:
    print(action)
    targetOrder = strats.query(f"strategy=={action['strategyId']}")
    targetOrderId = targetOrder['order'].values[0]
    targetOrderToken = FastSim.state()['orders'].query(f"id=={targetOrderId}")['tkn'].values[0]
    tokenAmount = Decimal(action['tokenAmount'])
    if not is_by_target:
        if action['orderIndex'] == 1:
            print('----1')
            FastSim.amm_buys(FastSim.carbon_pair.other(targetOrderToken),tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
        else:    
            print('----2')
            targetOrderId += 1
            FastSim.amm_buys(targetOrderToken,tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
        print(targetOrderToken, targetOrderId, tokenAmount)
    else:
        if action['orderIndex'] == 1:
            print('----3')
            FastSim.amm_sells(targetOrderToken,tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
        else:
            print('----4')
            targetOrderId += 1
            FastSim.amm_sells(FastSim.carbon_pair.other(targetOrderToken),tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
        print(FastSim.carbon_pair.other(targetOrderToken), targetOrderId, tokenAmount)
    

is_by_target False
{'strategyId': 0, 'orderIndex': 0, 'tokenAmount': '1000'}
----2
USDC 1 1000


In [10]:
FastSim.state()['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,None,1999.60467,ETH,1000.00000,USDC,ETHUSDC,1,1,0.50010,USDC per ETH
0,0,0,A,AMM sells 2000ETH buys 1000USDC,True,True,None,1999.60467,ETH,1000.00000,USDC,ETHUSDC,[1],1,0.500099,USDC per ETH


In [11]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

final_orders = FastSim.state()['orders']
final_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,865192.00000,865192.00000,USDC,False,9.00000,8.00000,9.00000,USDC per ETH,1
1,1,ETHUSDC,ETH,2962962.00000,2960962.39533,ETH,False,0.50000,1.00000,0.50020,USDC per ETH,0
2,2,ETHUSDC,USDC,987648.00000,987648.00000,USDC,False,10.00000,9.00000,10.00000,USDC per ETH,3
3,3,ETHUSDC,ETH,3950616.00000,3950616.00000,ETH,False,0.33333,0.50000,0.33333,USDC per ETH,2
4,4,ETHUSDC,USDC,1111104.00000,1111104.00000,USDC,False,11.00000,10.00000,11.00000,USDC per ETH,5
5,5,ETHUSDC,ETH,4938270.00000,4938270.00000,ETH,False,0.25000,0.33333,0.25000,USDC per ETH,4
6,6,ETHUSDC,USDC,1234560.00000,1234560.00000,USDC,False,12.00000,11.00000,12.00000,USDC per ETH,7
7,7,ETHUSDC,ETH,5925924.00000,5925924.00000,ETH,False,0.20000,0.25000,0.20000,USDC per ETH,6
8,8,ETHUSDC,USDC,1358016.00000,1358016.00000,USDC,False,13.00000,12.00000,13.00000,USDC per ETH,9
9,9,ETHUSDC,ETH,6913578.00000,6913578.00000,ETH,False,0.16667,0.20000,0.16667,USDC per ETH,8


In [12]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
compare_results = pd.concat([strats[['newLiquidity', 'newMarginalRate']].reset_index(drop=True), final_orders[['tkn', 'y', 'p_marg']].reset_index(drop=True)], axis=1)
compare_results.columns = ['json_liquidity', 'json_price', 'tkn', 'sim_liquidity', 'sim_price']
compare_results = pd.concat([compare_results.reset_index(drop=True), strats[['delta_liquidity']].reset_index(drop=True), (final_orders['y'] - initial_orders['y']).reset_index(drop=True)], axis=1)

compare_results = compare_results[['tkn','json_price', 'sim_price', 'json_liquidity', 'sim_liquidity', 'delta_liquidity', 'y']]
compare_results.loc[:,'sim_price'] = [1/compare_results.sim_price[i] if i%2==1 else compare_results.sim_price[i] for i in compare_results.index]
# compare_results = compare_results.astype(float)
compare_results

,tkn,json_price,sim_price,json_liquidity,sim_liquidity,delta_liquidity,y
0,USDC,9.000000000000,9.00000,865192,865192.00000,1000,1000.00000
1,ETH,1.999209661553,1.99921,2960963,2960962.39533,-1999,-1999.60467
2,USDC,10.000000000000,10.00000,987648,987648.00000,0,0.00000
3,ETH,3.000000000000,3.00000,3950616,3950616.00000,0,0.00000
4,USDC,11.000000000000,11.00000,1111104,1111104.00000,0,0.00000
5,ETH,4.000000000000,4.00000,4938270,4938270.00000,0,0.00000
6,USDC,12.000000000000,12.00000,1234560,1234560.00000,0,0.00000
7,ETH,5.000000000000,5.00000,5925924,5925924.00000,0,0.00000
8,USDC,13.000000000000,13.00000,1358016,1358016.00000,0,0.00000
9,ETH,6.000000000000,6.00000,6913578,6913578.00000,0,0.00000


In [13]:
bm_json

[{'strategies': [[{'liquidity': '864192',
     'lowestRate': '8.000000000000',
     'highestRate': '9.000000000000',
     'marginalRate': '9.000000000000',
     'newLiquidity': '865192',
     'newMarginalRate': '9.000000000000'},
    {'liquidity': '2962962',
     'lowestRate': '1.000000000000',
     'highestRate': '2.000000000000',
     'marginalRate': '2.000000000000',
     'newLiquidity': '2960963',
     'newMarginalRate': '1.999209661553'}],
   [{'liquidity': '987648',
     'lowestRate': '9.000000000000',
     'highestRate': '10.000000000000',
     'marginalRate': '10.000000000000',
     'newLiquidity': '987648',
     'newMarginalRate': '10.000000000000'},
    {'liquidity': '3950616',
     'lowestRate': '2.000000000000',
     'highestRate': '3.000000000000',
     'marginalRate': '3.000000000000',
     'newLiquidity': '3950616',
     'newMarginalRate': '3.000000000000'}],
   [{'liquidity': '1111104',
     'lowestRate': '10.000000000000',
     'highestRate': '11.000000000000',
     'm

In [14]:
json_strats = {}
sub_dict_list = []
for i in initial_orders.index[::2]:
    sub_dict0 = {}
    sub_dict0['liquidity'] = str(initial_orders.y[i])
    sub_dict0['lowestRate'] = str(initial_orders.p_end[i])
    sub_dict0['highestRate'] = str(initial_orders.p_start[i])
    sub_dict0['marginalRate'] = str(initial_orders.p_marg[i])
    sub_dict0['newLiquidity'] = str(final_orders.y[i])
    sub_dict0['newMarginalRate'] = str(final_orders.p_marg[i])
    
    sub_dict1 = {}
    sub_dict1['liquidity'] = str(initial_orders.y[i+1])
    sub_dict1['lowestRate'] = str(1/initial_orders.p_end[i+1])
    sub_dict1['highestRate'] = str(1/initial_orders.p_start[i+1])
    sub_dict1['marginalRate'] = str(1/initial_orders.p_marg[i+1])
    sub_dict1['newLiquidity'] = str(final_orders.y[i+1])
    sub_dict1['newMarginalRate'] = str(1/final_orders.p_marg[i+1])
    sub_dict_list += [[sub_dict0,sub_dict1]]
json_strats['strategies'] = sub_dict_list
json_strats

{'strategies': [[{'liquidity': '864192.0',
    'lowestRate': '8.0',
    'highestRate': '9.0',
    'marginalRate': '9.0',
    'newLiquidity': '865192.0',
    'newMarginalRate': '9.0'},
   {'liquidity': '2962962.0',
    'lowestRate': '1.0',
    'highestRate': '2.0',
    'marginalRate': '2.0000000000000004',
    'newLiquidity': '2960962.395328',
    'newMarginalRate': '1.9992094225092842'}],
  [{'liquidity': '987648.0',
    'lowestRate': '9.0',
    'highestRate': '10.0',
    'marginalRate': '10.000000000000002',
    'newLiquidity': '987648.0',
    'newMarginalRate': '10.000000000000002'},
   {'liquidity': '3950616.0',
    'lowestRate': '2.0',
    'highestRate': '3.0',
    'marginalRate': '2.9999999999999996',
    'newLiquidity': '3950616.0',
    'newMarginalRate': '2.9999999999999996'}],
  [{'liquidity': '1111104.0',
    'lowestRate': '10.0',
    'highestRate': '11.0',
    'marginalRate': '11.0',
    'newLiquidity': '1111104.0',
    'newMarginalRate': '11.0'},
   {'liquidity': '4938270.0'

In [15]:
bm_json[setup_index]['actions']

[{'strategyId': 0, 'orderIndex': 0, 'tokenAmount': '1000'}]

In [16]:
print(f'is_by_target {is_by_target}')
print(targetOrderToken, targetOrderToken==FastSim.carbon_pair.basetoken, action['orderIndex'])

is_by_target False
USDC False 0


In [17]:
json_actions = {}
sub_actions_list = []

trade_a = FastSim.amm_sells('ETH', 1000, execute=True) # route_trade_by_target
# trade_a = FastSim.amm_buys('ETH', 100000000, execute=True) # route_trade_by_source

if trade_a['success'] & trade_a['trades'].exec.all():
    subdf = trade_a['trades'].query('subid!="A"').copy()
    subdf.reset_index(inplace=True, drop=True)
    trade_is_by_target = trade_a['is_by_target']
    tkn = trade_a['tkn']
    if trade_is_by_target & (FastSim.carbon_pair.basetoken==tkn):
        orderIndex = 0
    elif trade_is_by_target & (FastSim.carbon_pair.basetoken!=tkn):
        orderIndex = 1
    elif (not trade_is_by_target) & (FastSim.carbon_pair.basetoken==tkn):
        orderIndex = 1
    elif (not trade_is_by_target) & (FastSim.carbon_pair.basetoken!=tkn):
        orderIndex = 0
    else:
        print('Failed')
    print(f'trade_is_by_target: {trade_is_by_target}')

    for i in subdf.index:
        sub_actions = {}
        sub_actions['strategyId'] = subdf.routeix[i]//2
        sub_actions['orderIndex'] = orderIndex
        if trade_is_by_target:
            sub_actions['tokenAmount'] = subdf.amt1[i]
        else:
            sub_actions['tokenAmount'] = subdf.amt2[i]
        sub_actions_list += [sub_actions]
        
        
json_actions['actions'] = sub_actions_list
json_actions

trade_is_by_target: True


{'actions': [{'strategyId': 9, 'orderIndex': 0, 'tokenAmount': 1000.0}]}

In [18]:
json_simulator = []
simulator_setup = {**json_strats, **json_actions}
json_simulator += [simulator_setup]
json_simulator

[{'strategies': [[{'liquidity': '864192.0',
     'lowestRate': '8.0',
     'highestRate': '9.0',
     'marginalRate': '9.0',
     'newLiquidity': '865192.0',
     'newMarginalRate': '9.0'},
    {'liquidity': '2962962.0',
     'lowestRate': '1.0',
     'highestRate': '2.0',
     'marginalRate': '2.0000000000000004',
     'newLiquidity': '2960962.395328',
     'newMarginalRate': '1.9992094225092842'}],
   [{'liquidity': '987648.0',
     'lowestRate': '9.0',
     'highestRate': '10.0',
     'marginalRate': '10.000000000000002',
     'newLiquidity': '987648.0',
     'newMarginalRate': '10.000000000000002'},
    {'liquidity': '3950616.0',
     'lowestRate': '2.0',
     'highestRate': '3.0',
     'marginalRate': '2.9999999999999996',
     'newLiquidity': '3950616.0',
     'newMarginalRate': '2.9999999999999996'}],
   [{'liquidity': '1111104.0',
     'lowestRate': '10.0',
     'highestRate': '11.0',
     'marginalRate': '11.0',
     'newLiquidity': '1111104.0',
     'newMarginalRate': '11.0'}

In [19]:
json_simulator[0]['actions']

[{'strategyId': 9, 'orderIndex': 0, 'tokenAmount': 1000.0}]

In [20]:
bm_json[0]['actions']

[{'strategyId': 0, 'orderIndex': 0, 'tokenAmount': '1000'}]

In [21]:
FastSim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,865192.00000,865192.00000,USDC,False,9.00000,8.00000,9.00000,USDC per ETH,1
1,1,ETHUSDC,ETH,2962962.00000,2960962.39533,ETH,False,0.50000,1.00000,0.50020,USDC per ETH,0
2,2,ETHUSDC,USDC,987648.00000,987648.00000,USDC,False,10.00000,9.00000,10.00000,USDC per ETH,3
3,3,ETHUSDC,ETH,3950616.00000,3950616.00000,ETH,False,0.33333,0.50000,0.33333,USDC per ETH,2
4,4,ETHUSDC,USDC,1111104.00000,1111104.00000,USDC,False,11.00000,10.00000,11.00000,USDC per ETH,5
5,5,ETHUSDC,ETH,4938270.00000,4938270.00000,ETH,False,0.25000,0.33333,0.25000,USDC per ETH,4
6,6,ETHUSDC,USDC,1234560.00000,1234560.00000,USDC,False,12.00000,11.00000,12.00000,USDC per ETH,7
7,7,ETHUSDC,ETH,5925924.00000,5925924.00000,ETH,False,0.20000,0.25000,0.20000,USDC per ETH,6
8,8,ETHUSDC,USDC,1358016.00000,1358016.00000,USDC,False,13.00000,12.00000,13.00000,USDC per ETH,9
9,9,ETHUSDC,ETH,6913578.00000,6913578.00000,ETH,False,0.16667,0.20000,0.16667,USDC per ETH,8


In [22]:
19//2

9

In [23]:
trade_a['trades'].query('subid=="A"')

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1,1,A,AMM sells 1000ETH buys 91USDC,True,True,None,1000.00000,ETH,90.90945,USDC,ETHUSDC,[19],1,0.09091,USDC per ETH


In [24]:
trade_a

{'success': True,
 'trades':    uid  id subid                           note   aggr  exec limitfail  \
 0  1.0   1     0                      route #19  False  True      None   
 0    1   1     A  AMM sells 1000ETH buys 91USDC   True  True      None   
 
         amt1 tkn1     amt2  tkn2     pair routeix  nroutes   price  \
 0 1000.00000  ETH 90.90945  USDC  ETHUSDC      19        1 0.09091   
 0 1000.00000  ETH 90.90945  USDC  ETHUSDC    [19]        1 0.09091   
 
          p_unit  
 0  USDC per ETH  
 0  USDC per ETH  ,
 'is_by_target': True,
 'tkn': 'ETH'}